In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
!pwd
os.chdir(r'/content/drive/MyDrive/Projects/Constitutions')

/content/drive/My Drive/Projects/Constitutions


***If you already have the packages jump this code***

In [7]:
!pwd
!pip uninstall spacy
!pip install spacy
!pip install spacy-universal-sentence-encoder
!pip3 install pickle5
!pip install nltk
!pip install gensim

/content/drive/MyDrive/Projects/Constitutions
Found existing installation: spacy 3.2.2
Uninstalling spacy-3.2.2:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.7/dist-packages/spacy-3.2.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/spacy/*
Proceed (y/n)? n


In [8]:
!pip install pickle5

**Making necessary imports**

In [9]:
import pickle5 as pickle
import logging
import warnings
import os

import pandas as pd
# import spacy
import spacy_universal_sentence_encoder as spacy_USE
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess

In [ ]:
#import pickle5 as pickle
#import pandas as pd

In [ ]:
"""From LDA.py we take open the saved DataFrame. Remember that Text data
have already been processed in this DataFrame (applied Lemmatization,
removing stopwords...)."""

folder = r'./df_total_all_set.pkl'
with open(folder, "rb") as fh:
  df = pickle.load(fh)

In [ ]:
# Not important to understand, make it easier to see running code
warnings.filterwarnings("ignore", category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

In [ ]:
df = df.sort_values(by=['country', 'year'])
text = df.constitution_lemma.to_list()

**It's important to remove all stopwords that for some reason Spacy can't remove. In the next step you can see that spacy will remove all stopwords words it finds, but after some runs in the program, we saw that this process was not perfect for our application (although great).**

In [ ]:
gensim_stop = STOPWORDS
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'vi', 'vii', 'viii',
                   'xi', 'xii', 'xv', 'xvii', '+', 'shall', 'iii', '*'', ii',
                   '©', 'bibliography', '<', '>', '/', 'iv', 'ix', 'may',
                   'constitution', 'country', 'shall'])
stop_words.extend(gensim_stop)
stop_words.extend(['sub', 'clause', 'article'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

def sent_to_words(sentences):
    """Do a simple tokenization."""
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(text))

# Define functions for stopwords
def remove_stopwords(texts):
    """Remove stopwords."""
    return [[word for word in simple_preprocess(str(doc)) if word not in
             stop_words] for doc in texts]

# Applying the functions just defined
data_words_nostops = remove_stopwords(data_words)

In [ ]:
processed_text = [' '.join(item) for item in data_words_nostops]

In [ ]:
print(processed_text[0:5])

['afghanistan completely free independent administration domestic foreign affair parts area authority majesty king treat single unit discrimination different religion afghanistan sacred religion islam follower religion jews hindus reside afghanistan entitled protection state prode disturb public peace kabul capital afghanistan people afghanistan entitle receive equal treatment government people kabul entitled special prilege etended people city llage ew etraordinary serce render cause progress independence afghan nation majesty king noble nation afghanistan pledge royal succession line principle male inheritance selection majesty people afghanistan majesty king ascend throne pledge nobles people rule accordance principle enunciate sharia protect independence remain faithful nation majesty king servant protector true religion islam ruler king afghanistan affair administer minister government select appoint king minister responsible ministry king responsible mention king khutba friday pr

**Important part**

In [ ]:
def get_spacy_list(texts):
    """
    Take the list of texts and turn into nlp spacy list.

    Also, it removes stopwords words from the data, where spacy transformer
    pipeline have the definition of stopwords.
    (See: https://spacy.io/usage/spacy-101).

    Parameters
    ----------
    texts : list
        DESCRIPTION. List of texts

    Returns
    -------
    list of nlp pipeline for each text.
    """
    texts_out = []
    nlp = spacy_USE.load_model('en_use_lg')
    for text in texts:
        doc = nlp(text)
        texts_out.append([token.text for token in doc if not token.is_stop])

    text_nlp = [' '.join(item) for item in texts_out]
    text_pipe = [nlp(tex) for tex in text_nlp]

    return text_pipe


text_pipe = get_spacy_list(processed_text)

df['nlp_doc'] = text_pipe
df_final = df[['country', 'year', 'nlp_doc']]

In [ ]:
df_final.to_pickle('df_USE.pkl')

**This step is only made to reduce memory usage. We could create order program file, but we wanted to maintain just one.**

In [41]:
df_final = pd.read_pickle('df_USE.pkl')

**Back in business**

In [42]:
base = df_final.set_index('country')
print(base.head())

             year                                            nlp_doc
country                                                             
Afghanistan  1923  (afghanistan, completely, free, independent, a...
Afghanistan  1931  (general, principles, faith, afghanistan, sacr...
Afghanistan  1964  (god, almighty, organize, national, life, afgh...
Afghanistan  1977  (nation, society, course, history, state, cons...
Afghanistan  1987  (allah, beneficent, merciful, beloved, homelan...


In [43]:
base = base.drop('year', axis=1)


In [44]:
base.shape

(617, 1)

In [ ]:
# df_kenya = base.iloc[299:301]
#kenya_0 = df_kenya.iloc[0,0]
#kenya_1 = df_kenya.iloc[1,0]
#result12_2 = kenya_0.similarity(kenya_1)

**In this part we create the final data structure that we want to process**

In [15]:
def get_dfs(dataframe):
  """Create a dictonary of dataframes for all countries."""
  
  df_group = dataframe.groupby(['country'])
  dfs = dict(tuple(df_group))

  return dfs

In [46]:
dfs = get_dfs(base)

In [47]:
number = 0
for country in dfs:
  number += dfs[country].shape[0]
print(number)

617


In [48]:
def cosine_similarity_all(dictionary):
  """Compute the cosine similarity between to Spacy_pipeline processed texts.

  Very Heavy program.
  """

  similarity_sequence = []
  for country in dictionary:
    similarity_sequence.append(1.0)
    for i in range(len(dictionary[country])-1):
      sim = dictionary[country].iloc[i, 0].similarity(
            dictionary[country].iloc[i+1, 0])
      similarity_sequence.append(sim)
  
  return similarity_sequence

In [21]:
result = cosine_similarity_all(dfs)

Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB



In [49]:
df_sequence = df_final.drop('nlp_doc', axis=1)
df_sequence['similarity'] = result
print(df_sequence.head())

       country  year  similarity
0  Afghanistan  1923    1.000000
1  Afghanistan  1931    0.944877
2  Afghanistan  1964    0.878636
3  Afghanistan  1977    0.955040
4  Afghanistan  1987    0.971161


In [50]:
df_sequence.to_csv('similarity_USE.csv')

# **Alert**
**This program above (to measure similarity) is heavy even for 35 gb of ram. We've tried to solve this issue by slicing the list and making the process separately for each part. You don't need to run any of this, it was only created to see which constitutionals were to heavy for the above program to run smoothly in a budget computer.**

In [18]:
keys = list(dfs.keys())
print(keys)
print(len(keys))

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Austria Hungary', 'Azerbaijan', 'Baden', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Bavaria', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'BosniaAndHerzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Caribbean Charter', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Czechoslovakia', 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'German Democratic Republic', 'Germany', 'Germany Federal Republic', 'Germany(Prussia)', 'Ghana', 'Great Colombia', 'Greece', 'Grenada',

In [ ]:
keys1 = keys[0:10]
keys2 = keys[10:20]
keys3 = keys[20:30]
keys4 = keys[30:40]
keys5 = keys[40:50]
keys6 = keys[50:60]
keys7 = keys[60:70]
keys8 = keys[70:80]
keys9 = keys[80:85]
keys10 = keys[85:90]
keys11 = keys[90:92]
keys12 = keys[92:93]
keys12_1 = keys[93:94]
keys12_2 = keys[94:95]
keys13 = keys[95:100]
keys14 = keys[100:110]
keys15 = keys[110:120]
keys16 = keys[120:130]
keys17 = keys[130:140]
keys18 = keys[140:150]
keys19 = keys[150:160]
keys20 = keys[160:170]
keys21 = keys[170:180]
keys22 = keys[180:190]
keys23 = keys[190:200]
keys24 = keys[200:210]

In [ ]:
dfs1 = {x:dfs[x] for x in keys1}
dfs2 = {x:dfs[x] for x in keys2}
dfs3 = {x:dfs[x] for x in keys3}
dfs4 = {x:dfs[x] for x in keys4}
dfs5 = {x:dfs[x] for x in keys5}
dfs6 = {x:dfs[x] for x in keys6}
dfs7 = {x:dfs[x] for x in keys7}
dfs8 = {x:dfs[x] for x in keys8}
dfs9 = {x:dfs[x] for x in keys9}
dfs10 = {x:dfs[x] for x in keys10}
dfs11 = {x:dfs[x] for x in keys11}
dfs12 = {x:dfs[x] for x in keys12}
dfs12_1 = {x:dfs[x] for x in keys12_1}
dfs12_2 = {x:dfs[x] for x in keys12_2}
dfs13 = {x:dfs[x] for x in keys13}
dfs14 = {x:dfs[x] for x in keys14}
dfs15 = {x:dfs[x] for x in keys15}
dfs16 = {x:dfs[x] for x in keys16}
dfs17 = {x:dfs[x] for x in keys17}
dfs18 = {x:dfs[x] for x in keys18}
dfs19 = {x:dfs[x] for x in keys19}
dfs20 = {x:dfs[x] for x in keys20}
dfs21 = {x:dfs[x] for x in keys21}
dfs22 = {x:dfs[x] for x in keys22}
dfs23 = {x:dfs[x] for x in keys23}
dfs24 = {x:dfs[x] for x in keys24}

In [ ]:
# print(dfs12['Jordan'].nlp_doc)

In [ ]:
def cosine_similarity_use(dictionary):
  """Compute the cosine similarity between to Spacy_pipeline processed texts.

  Very Heavy program.
  """

  similarity_sequence = []
  for country in dictionary:
    similarity_sequence.append(1.0)
    for i in range(len(dictionary[country])-1):
        sim = dictionary[country].iloc[i, 0].similarity(
            dictionary[country].iloc[i+1, 0])
        similarity_sequence.append(sim)
  
  return similarity_sequence

In [ ]:
result1 = cosine_similarity_use(dfs1)

In [ ]:
result2 = cosine_similarity_use(dfs2)

In [ ]:
result3 = cosine_similarity_use(dfs3)

In [ ]:
result4 = cosine_similarity_use(dfs4)

In [ ]:
result5 = cosine_similarity_use(dfs5)

In [ ]:
result6 = cosine_similarity_use(dfs6)

In [ ]:
result7 = cosine_similarity_use(dfs7)

In [ ]:
result8 = cosine_similarity_use(dfs8)

In [ ]:
result9 = cosine_similarity_use(dfs9)

In [ ]:
result10 = cosine_similarity_use(dfs10)

In [ ]:
result11 = cosine_similarity_use(dfs11)

In [ ]:
result12 = cosine_similarity_use(dfs12)

In [ ]:
result12_1 = cosine_similarity_use(dfs12_1)

**carefull: This cel runs out of memory even if it is just one country similarity.**

In [ ]:
result12_2 = cosine_similarity_use(dfs12_2)

In [ ]:
result13 = cosine_similarity_use(dfs13)

In [ ]:
result14 = cosine_similarity_use(dfs14)

In [ ]:
result15 = cosine_similarity_use(dfs15)

In [ ]:
result16 = cosine_similarity_use(dfs16)

In [ ]:
result17 = cosine_similarity_use(dfs17)

In [ ]:
result18 = cosine_similarity_use(dfs18)

In [ ]:
result19 = cosine_similarity_use(dfs19)

In [ ]:
result20 = cosine_similarity_use(dfs20)

In [ ]:
result21 = cosine_similarity_use(dfs21)

In [ ]:
result22 = cosine_similarity_use(dfs22)

In [ ]:
result23 = cosine_similarity_use(dfs23)

In [ ]:
result24 = cosine_similarity_use(dfs24)

In [ ]:
result12_2

In [ ]:
from itertools import chain
similarity_sequence = list(chain(result1, result2, result3, result4, result5,
                                 result6, result7, result8, result9, result10,
                                 result11, result12, result12_1, result12_2,
                                 result13, result14,
                                 result15, result16, result17, result18,
                                 result19, result20, result21, result22,
                                 result23, result24))

In [ ]:
df_sequence = df_final.drop('nlp_doc', axis=1)
df_sequence['similarity'] = similarity_sequence
print(df_sequence.head())